# Clean Data
In this notebook, we'll take the raw data and reorganize them so that we can use them for visualization and a predictive model. These are the files we will be working with:
```
incidents.csv                 List of gun violence incidents from January 2014 - March 2018
population.csv                Population from 2000 - 2017 intercensal estimates for July
other_crime_annual.csv        Annual sums of crimes by state from 2010 - 2016
income.csv                    Annual average personal income by state from 2009-2017         
annual_gun_deaths.csv         Annual deaths by guns (homicides only) by state from 1999-2013
alcohol.csv                   Annual alcohol consumption by state from 1977-2016
provisions.csv                List of provisions in place by each state for the years 1991-2017
election_results.csv          List of presidential election results by state from years 2000-2016
```
For more information on where these datasets came from, see the writeup. Our goal is to make several DataFrames which will be used later.

One is an overall DataFrame with statistics aggregated annually. This DataFrame will have entries from 2010-2016. This will be used for visualization.

Another is a feature DataFrame with monthly gun homicides from 2014-2017, each entry coupled with annual statistics from the previous year. This will be used later to make a model to predict gun violence trends.



In [1]:
# Numpy and pandas for manipulating the data
import numpy as np
import pandas as pd

In [2]:
daily_incidents_file = './data/raw/incidents.csv.gz' # zipped because file is too big
population_file = './data/raw/population.csv'
crime_file = './data/raw/other_crime_annual.csv'
income_file = './data/raw/income.csv'
annual_file = './data/raw/annual_gun_deaths.csv'
alcohol_file = './data/raw/alcohol.csv'
provisions_file = './data/raw/provisions.csv'
election_file = './data/raw/election_results.csv'

daily_incidents_df = pd.read_csv(daily_incidents_file, parse_dates=True, compression='gzip')
population_df = pd.read_csv(population_file, parse_dates=True, index_col=0)
annual_gun_deaths_df = pd.read_csv(annual_file, parse_dates=True)
crime_df = pd.read_csv(crime_file, parse_dates=True)
income_df = pd.read_csv(income_file, parse_dates=True)
alcohol_df = pd.read_csv(alcohol_file, parse_dates=True)
provisions_df = pd.read_csv(provisions_file, parse_dates=True)
election_df = pd.read_csv(election_file)

## Incidents File
### daily_incidents_df and population_df
Goals: 

    lat_long_df         Incidents with latitude and longitude coordinates (2014-2017)
    feature_df          Daily incidents per state, each with features from the previous year (2014-2017)
    by_date_total_df    Daily gun homicides per state indexed by date with states in columns
    by_date_norm_df     Same as by_date_total_df but normalized by population (gun homicides / 100000 people)

In [3]:
null_count = daily_incidents_df.isnull().sum()
null_count.sort_values()

incident_id                         0
date                                0
state                               0
city_or_county                      0
n_killed                            0
n_injured                           0
incident_url                        0
incident_url_fields_missing         0
incident_characteristics          327
source_url                        468
sources                           610
longitude                        7923
latitude                         7923
congressional_district          11945
address                         16497
participant_type                24864
participant_status              27627
state_senate_district           32336
participant_gender              36363
state_house_district            38773
participant_age_group           42120
notes                           81018
participant_age                 92299
n_guns_involved                 99452
gun_type                        99452
gun_stolen                      99499
participant_

There seems to be many missing values about the details of each incident. However, the 'state' and 'date' columns have no missing values, which is good for our objective (examining gun violence trends for each state).

In [4]:
# First let's make some columns that we'll need
daily_incidents_df['date'] = pd.to_datetime(daily_incidents_df['date']) # Turn the date into a datetime
daily_incidents_df['year'] = daily_incidents_df['date'].dt.year # Create a column for just the year

# Upon examining the data, it seems that records before 2014 have missing data, so we'll exclude them
daily_incidents_df = daily_incidents_df[daily_incidents_df['year'] >= 2014]

# Get number of casualties for each state, indexed by date
by_date_total_df = daily_incidents_df.groupby(['date', 'state'])['n_killed'].sum().unstack()
by_date_total_df = by_date_total_df.fillna(0) # Some days had no incidents (no entries). Fill with 0s

# Now we should normalize the number of casualties by state population. 
# We use population_df: data from US Census Bureau

# Transform on a resampled column: get population in that year from population_df
# multiply by 100,000 to get number of casualties per 100,000 people
def normalize_population(x):
    state_name = x.name
    year = str(x.index.year[0])
    population = population_df.loc[state_name, year]
    return x * 100000 / population

by_date_norm_df = by_date_total_df['2014':'2017'].resample('A').transform(normalize_population)
by_date_norm_df.head()

by_date_total_df.to_csv('./data/cleaned/by_date_total.csv')
by_date_norm_df.to_csv('./data/cleaned/by_date_norm.csv')

In [5]:
# Get latitude and longitude information and save it
lat_long_df = daily_incidents_df[['state', 'latitude', 'longitude', 'n_killed']].dropna()
lat_long_df.to_csv('./data/cleaned/lat_long.csv')

In [6]:
# Create a feature DataFrame to store all of our features (to be used later in our model)
feature_df = by_date_total_df.resample('M').sum().stack().reset_index()
feature_df.columns = ['next_date', 'state', 'next_deaths']
# Get the number of casualties normalized by population
# Next year (the year which we want to predict n_casualties)
feature_df['next_year'] = feature_df['next_date'].apply(lambda x: x.year)
feature_df['this_year'] = feature_df['next_year'] - 1
feature_df['population'] = feature_df[['state', 'this_year']]\
                            .apply(lambda x: population_df.loc[x['state'], str(x['this_year'])], axis=1)
    
# Reformat population data
population_df = population_df.stack().reset_index()
population_df.columns = ['state', 'year', 'population']
population_df['year'] = population_df['year'].astype(int)

# Clean annual_gun_deaths_df so we have observations in each row; note this has data from 1999-2013
annual_1999_2013_df = annual_gun_deaths_df.set_index('state').stack().reset_index()
annual_1999_2013_df.columns = ['state', 'year', 'gun_deaths']

# Sum incidents annually and make each row an observation w/ state, year, and number of incidents 
annual_2014_2017 = by_date_total_df[:'2017'].resample('A').sum().stack().reset_index()
annual_2014_2017['date'] = annual_2014_2017['date'].apply(lambda x: x.year)

# Rearrange columns and concat the dataframes
annual_2014_2017.columns = ['year', 'state', 'gun_deaths']
annual_2014_2017 = annual_2014_2017[['state', 'year', 'gun_deaths']]

# Note: later, we lose year 1999 when we merge with population since population_df goes from 2000-2017 only
annual_2000_2017_df = pd.concat([annual_1999_2013_df, annual_2014_2017])
annual_2000_2017_df['year'] = annual_2000_2017_df['year'].astype(int)

# Add population data to annual_2000_2017
annual_2000_2017_df = pd.merge(annual_2000_2017_df, population_df)
annual_2000_2017_df = annual_2000_2017_df.sort_values(['state', 'year'])

# Create a DataFrame that will hold all features aggregated annually from 2010-2016.
overall_2010_2016_df = pd.merge(annual_2000_2017_df, population_df).sort_values(['state','year'])

## Other Crime
We'll be working with this data:
    
    crime_df                 Annual crime data (2010-2016) from disastercenter.com/crime/  
    annual_gun_deaths_df     Annual gun homicides by state (1999-2013) from gunpolicy.org
And creating/updating these DataFrames:

    overall_2010_2016_df     DataFrame to hold annual statistics from 2010-2016 to be plotted later
    annual_2000_2017_df      DataFrame that holds annual gun death info from 2000-2017
    feature_df               Update with crime features

In [7]:
# Note: other_crime here is defined by crimes not including murder
crime_df['other_crime'] = crime_df[['rape_crime', 'robbery_crime', 'assault_crime', 
                                'burglary_crime', 'larceny_theft_crime', 'vehicle_theft_crime']].sum(axis=1)
crime_df = crime_df.drop(['population', 'index'], axis=1)
crime_df['year'] = crime_df['year'].astype(int)

overall_2010_2016_df = pd.merge(overall_2010_2016_df, crime_df)

# Let's update our feature dataframe with the crime rate
feature_df = pd.merge(feature_df, crime_df, left_on=['state','this_year'], right_on=['state', 'year'])
feature_df = feature_df.drop('year', axis=1)

In [14]:
crime_df

,state,year,violent_crime,property_crime,murder_crime,rape_crime,robbery_crime,assault_crime,burglary_crime,larceny_theft_crime,vehicle_theft_crime,other_crime
0,Alaska,2016,4537,20259,31,533,594,3379,3105,15535,1619,24765
1,Alaska,2015,4416,19094,30,436,576,3374,2852,14854,1388,23480
2,Alaska,2014,4412,20037,30,583,630,3169,2950,15565,1522,24419
3,Alaska,2013,4709,21211,34,657,623,3127,2917,16599,1695,25618
4,Alaska,2012,4684,20334,41,553,629,3243,3150,15445,1739,24759
5,Alaska,2011,5391,20806,59,648,761,3671,3511,15249,2046,25886
6,Alaska,2010,5966,24876,52,757,850,4011,4053,17766,3057,30494
7,Alabama,2016,18363,168828,275,1355,4864,11869,42484,115564,10780,186916
8,Alabama,2015,20166,173192,299,1370,4906,13591,51119,111411,10662,193059
9,Alabama,2014,21693,168878,342,1296,5020,15035,47481,111523,9874,190229


## Personal Income
We'll be working with this data:
    
    income_df               Personal income per capita (2009-2017) from US Bureau of Economic Analysis
Goals:
    
    overall_2010_2016_df    Update with personal income per capita
    feature_df              Update with personal income per capita

In [8]:
# Add income data
income_by_state_df = income_df.set_index('state').stack().reset_index()
income_by_state_df.columns = ['state', 'year', 'income']
income_by_state_df['year'] = income_by_state_df['year'].astype(int)

overall_2010_2016_df = pd.merge(overall_2010_2016_df, income_by_state_df)
feature_df = pd.merge(feature_df, income_by_state_df, left_on=['state', 'this_year'], right_on=['state', 'year'])
feature_df = feature_df.drop('year', axis=1)

## Alcohol
### alcohol_df
Goals:

    overall_2010_2016_df    update with avg total alcohol consumption 
    feature_df              update with all alcohol features 

In [9]:
# Merge the alcohol features into the overall and feature data frames
overall_2010_2016_df = pd.merge(overall_2010_2016_df, alcohol_df)
feature_df = pd.merge(feature_df, alcohol_df, left_on=['state', 'this_year'], right_on=['state', 'year'])
feature_df = feature_df.drop('year', axis=1)

## Provisions
### provisions_df
Goals:
    
    overall_2010_2016_df     update with total provision count
    feature_df               update with all provisions

In [10]:
# Update the overall_2010_2016_df with the total provisions
overall_2010_2016_df = pd.merge(overall_2010_2016_df, provisions_df)

# Update the feature_df with all of the provisions
feature_df = pd.merge(feature_df, provisions_df, left_on=['state', 'this_year'], right_on=['state', 'year'])
feature_df = feature_df = feature_df.drop('year', axis=1)

## Election Results
### election_df
Goals:

    annual_2000_2017_df      Update with election results
    overall_2010_2016_df     Update with election results
    feature_df               Update with election results from previous year

In [11]:
# For simplicity, let's expand the election_df s.t. it has entries for non-election years with values
# from the previous election. This way, we can just merge them with the other dfs later
election_expand = []
for i in range(4):
    temp_df = election_df.copy()
    temp_df['year'] = election_df['year'] + i
    election_expand.append(temp_df)

election_df = pd.concat(election_expand)

# Update each of the dataframes with the election information
annual_2000_2017_df = pd.merge(annual_2000_2017_df, election_df)

overall_2010_2016_df = pd.merge(overall_2010_2016_df, election_df)

feature_df = pd.merge(feature_df, election_df, left_on=['state', 'this_year'], right_on=['state', 'year'])
feature_df = feature_df.drop('year', axis=1)

In [12]:
# Add features normalized by population to the annual and overall dataframes
overall_population = overall_2010_2016_df['population']
overall_2010_2016_df['gun_deaths_norm'] = overall_2010_2016_df['gun_deaths'] / overall_population * 100000
overall_2010_2016_df['other_crime_norm'] = overall_2010_2016_df['other_crime'] / overall_population * 100000

annual_population = annual_2000_2017_df['population']
annual_2000_2017_df['gun_deaths_norm'] = annual_2000_2017_df['gun_deaths'] / annual_population * 100000

In [13]:
# Save annual, overall and feature DataFrames
annual_2000_2017_df.to_csv('./data/cleaned/annual_gun.csv')
overall_2010_2016_df.to_csv('./data/cleaned/overall.csv')
feature_df.to_csv('./data/cleaned/feature.csv')

To recap, here are the DataFrames that were saved:

    lat_long_df                Incidents with latitude and longitude coordinates (2014-2017)
    by_date_tot_df             Gun homicides aggregated daily and by state (2014-2017)
    by_date_norm_df            Same as by_date_tot_df, but normalized by state population
    annual_2000_2017           Annual gun homicides with population info(2000-2017)
    overall_2010_2016_df       Annual features from (2010-2016)
    feature_df                 Gun homicides aggregated monthly and by state, paired with annual features from
                               the previous year (2014-2017)